In [1]:
import pyximport
pyximport.install()

(None, <pyximport.pyximport.PyxImporter at 0x7fe1b7e7b7f0>)

In [2]:
%load_ext line_profiler

In [3]:
import utils
import utils_opt_ver1
import utils_opt_ver2
import sknn_opt
import sknn_opt_ver2
import sknn 
import pandas as pd
import numpy as np

In [4]:
FOLDER_TRAIN = 'data/train/short_ver/'
FOLDER_TEST = 'data/test/'
OUTPUT_FOLDER = 'output/'

## Baseline Original Code

In [5]:
%time train, test = utils.load_dataset(FOLDER_TRAIN, FOLDER_TEST, feather=True)

load_dataset
 -- loaded in: 0.36977624893188477s
 -- filtered in: 0.503352165222168s
 -- actions:  664712
 -- items:  170089
 -- lists:  10000
CPU times: user 457 ms, sys: 180 ms, total: 637 ms
Wall time: 558 ms


In [6]:
rec = sknn.SessionKNN(2000, 0, idf_weight=1, folder=OUTPUT_FOLDER)

In [10]:
%timeit -r 10 rec.train(train, test)

 -- finished 100000 of 664712 rows in 0.6771631240844727s
 -- finished 200000 of 664712 rows in 1.334989070892334s
 -- finished 300000 of 664712 rows in 2.0025322437286377s
 -- finished 400000 of 664712 rows in 2.6726672649383545s
 -- finished 500000 of 664712 rows in 3.3400282859802246s
 -- finished 600000 of 664712 rows in 4.040913343429565s
 -- finished 100000 of 664712 rows in 0.650623083114624s
 -- finished 200000 of 664712 rows in 1.3203508853912354s
 -- finished 300000 of 664712 rows in 1.9840829372406006s
 -- finished 400000 of 664712 rows in 2.649099111557007s
 -- finished 500000 of 664712 rows in 3.314030885696411s
 -- finished 600000 of 664712 rows in 3.9873299598693848s
 -- finished 100000 of 664712 rows in 0.6659860610961914s
 -- finished 200000 of 664712 rows in 1.3258891105651855s
 -- finished 300000 of 664712 rows in 1.9871599674224854s
 -- finished 400000 of 664712 rows in 2.6597330570220947s
 -- finished 500000 of 664712 rows in 3.3332200050354004s
 -- finished 600000

In [17]:
test_data = list(zip(test['playlists']['playlist_id'], test['playlists']['name'], 
                     test['playlists']['num_tracks'], test['playlists']['num_samples']))
pid, name, ntracks, nsamples = test_data[5000]
num_hidden = ntracks - nsamples

In [18]:
tracks = test['actions'][test['actions']['playlist_id']==pid]['track_id'].values
artist_ids = test['actions'][test['actions']['playlist_id']==pid]['artist_id'].values

In [19]:
rec.predict(name, tracks, playlist_id=pid, artists=artist_ids, num_hidden=num_hidden) 

,track_id,confidence
0,1257,1.008693
1,6941,0.877109
2,681,0.875561
3,2155,0.875269
4,848,0.840196
...,...,...
495,18683,0.176009
496,4779,0.175825
497,7293,0.175498
498,2192,0.175038


## Optimization Attempts

### Ver1

In [ ]:
# %lprun -f rec.train rec.train(train, test)

In [8]:
%time train, test = utils_opt_ver1.load_dataset(FOLDER_TRAIN, FOLDER_TEST, feather=True)

load_dataset
 -- loaded in: 0.33950090408325195s
 -- filtered in: 0.4713008403778076s
(' -- actions: ', 664712)
(' -- items: ', 170089)
(' -- lists: ', 10000)
CPU times: user 434 ms, sys: 133 ms, total: 567 ms
Wall time: 545 ms


In [9]:
rec = sknn_opt.SessionKNN(2000, 0, idf_weight=1, folder=OUTPUT_FOLDER)

In [10]:
%time rec.train(train, test)

CPU times: user 1.41 s, sys: 221 ms, total: 1.63 s
Wall time: 5.94 s


In [11]:
test_data = list(zip(test['playlists']['playlist_id'], test['playlists']['name'], 
                     test['playlists']['num_tracks'], test['playlists']['num_samples']))
pid, name, ntracks, nsamples = test_data[5000]
num_hidden = ntracks - nsamples

tracks = test['actions'][test['actions']['playlist_id']==pid]['track_id'].values
artist_ids = test['actions'][test['actions']['playlist_id']==pid]['artist_id'].values

rec.predict(name, tracks, playlist_id=pid, artists=artist_ids, num_hidden=num_hidden) 

### Ver2

In [11]:
%time train, test = utils.load_dataset(FOLDER_TRAIN, FOLDER_TEST, feather=True)

load_dataset
 -- loaded in: 0.36095595359802246s
 -- filtered in: 0.498060941696167s
 -- actions:  664712
 -- items:  170089
 -- lists:  10000
CPU times: user 441 ms, sys: 138 ms, total: 579 ms
Wall time: 553 ms


In [12]:
rec = sknn_opt_ver2.SessionKNN(2000, 0, idf_weight=1, folder=OUTPUT_FOLDER)

In [13]:
%timeit -r 10 rec.train(train, test)

4.02 s ± 77.3 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [ ]:
test_data = list(zip(test['playlists']['playlist_id'], test['playlists']['name'], 
                     test['playlists']['num_tracks'], test['playlists']['num_samples']))
pid, name, ntracks, nsamples = test_data[5000]
num_hidden = ntracks - nsamples

tracks = test['actions'][test['actions']['playlist_id']==pid]['track_id'].values
artist_ids = test['actions'][test['actions']['playlist_id']==pid]['artist_id'].values

rec.predict(name, tracks, playlist_id=pid, artists=artist_ids, num_hidden=num_hidden) 